### Web scrape valid surf rides from the Smartfin website: 

In [30]:
#Imports 
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import re
import time

print("Done.")

Done.


### Scrape verified surf sessions into a dataframe (can add more later):

In [31]:
# Start by just looking at my surf sessions in SD (near CDIP buoy): 

# 15218 - First VIRB filmed session - Oct. 24, 2018
# 15669 - Second VIRB filmed session - Nov. 7, 2018
# 15692 - Third VIRB filmed session - Nov. 9, 2018
# 15686 - Fourth VIRB filmed session - Nov. 11, 2018

ride_ids = ["15896"]
#ride_ids = [15218, 15669, 15692, 15686]

In [3]:
#Get urls for surf sessions in Scripps beach area that most likely correspond to surfing time intervals: 
url_45_60 = 'https://surf.smartfin.org/advanced_search/?northEastLat=32.6&northEastLon=-117.24&southWestLat=32.9&southWestLon=-117.33&dateTimeBegin=&dateTimeEnd=&timeZone=PDT&durationMin=45&durationMax=60&sensorTypeWave=true#searchResultsContainer'
url_61_80 = 'https://surf.smartfin.org/advanced_search/?northEastLat=32.6&northEastLon=-117.24&southWestLat=32.9&southWestLon=-117.33&dateTimeBegin=&dateTimeEnd=&timeZone=PDT&durationMin=61&durationMax=80&sensorTypeWave=true#searchResultsContainer'
url_81_100 = 'https://surf.smartfin.org/advanced_search/?northEastLat=32.6&northEastLon=-117.24&southWestLat=32.9&southWestLon=-117.33&dateTimeBegin=&dateTimeEnd=&timeZone=PDT&durationMin=81&durationMax=100&sensorTypeWave=true#searchResultsContainer'
url_101_130 = 'https://surf.smartfin.org/advanced_search/?northEastLat=32.6&northEastLon=-117.24&southWestLat=32.9&southWestLon=-117.33&dateTimeBegin=&dateTimeEnd=&timeZone=PDT&durationMin=101&durationMax=130&sensorTypeWave=true#searchResultsContainer'

#urls = [url_61_80]
#urls = [url_45_60, url_61_80, url_81_100, url_101_130]

In [4]:
# Define a function that will parse rows from data tables on CDIP's website:
def parse_rows(row_number):
    name_string = ""
    name_list = []
    for t in tr_elements[row_number]:
        name = t.text_content()
        for i in name: 
            name_string += i
    #print(name_string)

    # Create a string from the values
    name_string = name_string.split(" ")
    
    # Remove all spaces from the list
    for i in name_string: 
        if len(i) > 0:
            name_list.append(i)
      
    # Ensure that time stays with 'Date (UTC)'' header
    name_list[0] = name_list[0] + " " +  name_list[1]
    name_list.pop(1)
    
    return name_list

In [5]:
# Iterate over each url and get all ride_ids located at that url: 
ride_ids = []
for u in urls: 
    
    # Create a handle, page, to handle the contents of the website
    page = requests.get(u)

    # Store the contents of the website under doc
    doc = lh.fromstring(page.content)

    # Parse data stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')

    # Retrieve all of the ride ids: 
    for j in range(1, len(tr_elements)):
        data = parse_rows(j)
        data = str(data[0]).strip("\n")
        if int(data) > 10000: 
            ride_ids.append(data)

print(len(ride_ids))
print(ride_ids[:20])
print(ride_ids[-10:])

265
[' 16302', ' 16301', ' 16298', ' 16290', ' 16262', ' 16243', ' 16237', ' 16236', ' 16196', ' 16195', ' 16192', ' 16168', ' 16161', ' 16148', ' 16073', ' 16072', ' 16000', ' 15972', ' 15947', ' 15859']
[' 10191', ' 10157', ' 10085', ' 10084', ' 10068', ' 10060', ' 10040', ' 10036', ' 10015', ' 10008']


In [6]:
# Fin ID Scraper: 

ride_url_base = 'http://surf.smartfin.org/ride/'
str_id_csv = 'img id="temperatureChart" class="chart" src="' # Look for this text in the HTML contents in fcn below

def read_csv_from_ride(ride): 
    ride_url = ride_url_base + str(ride)
    html_contents = requests.get(ride_url).text # Get contents of ride_url
    loc_csv_id = html_contents.find(str_id_csv) # Find CSV identifier 
    
    # Different based on whether user logged in with FB or Google
    offset_googleOAuth = [46, 114]
    offset_facebkOAuth = [46, 112]
    if html_contents[loc_csv_id+59] == 'f': # Facebook login
        off0 = offset_facebkOAuth[0]
        off1 = offset_facebkOAuth[1]
    else: # Google login
        off0 = offset_googleOAuth[0]
        off1 = offset_googleOAuth[1]
        
    csv_id_longstr = html_contents[loc_csv_id+off0:loc_csv_id+off1]
        
    # Stitch together full URL for CSV
        
    motion_csv_url = 'https://surf.smartfin.org/'+csv_id_longstr+'Motion.CSV'

    #print(motion_csv_url)

    motion_df_small = pd.read_csv(motion_csv_url)
    #print(len(motion_df_small))
        
        
    return motion_df_small

In [7]:
#ride_ids = ride_ids[:10]

In [8]:
#%% Fin ID scraper
# Input fin ID, get all ride IDs
# base URL to which we'll append given fin IDs
fin_url_base = 'http://surf.smartfin.org/fin/'

# Look for the following text in the HTML contents in fcn below
str_id_ride = 'rideId = \'' # backslash allows us to look for single quote
str_id_date = 'var date = \'' # backslash allows us to look for single quote

#%% Ride ID scraper
# Input ride ID, get ocean and motion CSVs
# Base URL to which we'll append given ride IDs
ride_url_base = 'https://surf.smartfin.org/ride/'

# Look for the following text in the HTML contents in fcn below
str_id_csv = 'img id="temperatureChart" class="chart" src="' 

def get_csv_from_ride_id(rid):
    # Build URL for each individual ride
    ride_url = ride_url_base+str(rid)
    #print(ride_url)
    
    # Get contents of ride_url
    html_contents = requests.get(ride_url).text
    
    # Find CSV identifier 
    loc_csv_id = html_contents.find(str_id_csv)
    
    # Different based on whether user logged in with FB or Google
    offset_googleOAuth = [46, 114]
    offset_facebkOAuth = [46, 112]
    if html_contents[loc_csv_id+59] == 'f': # Facebook login
        off0 = offset_facebkOAuth[0]
        off1 = offset_facebkOAuth[1]
    else: # Google login
        off0 = offset_googleOAuth[0]
        off1 = offset_googleOAuth[1]
        
    csv_id_longstr = html_contents[loc_csv_id+off0:loc_csv_id+off1]
        
    # Stitch together full URL for CSV
    if ("media" in csv_id_longstr) & ("Calibration" not in html_contents): # other junk URLs can exist and break everything
        
        ocean_csv_url = 'https://surf.smartfin.org/'+csv_id_longstr+'Ocean.CSV'
        motion_csv_url = 'https://surf.smartfin.org/'+csv_id_longstr+'Motion.CSV'
        
        print(motion_csv_url)
        
        # Go to ocean_csv_url and grab contents (theoretically, a CSV)
        ocean_df_small = pd.read_csv(ocean_csv_url, parse_dates = [0])
        elapsed_timedelta = (ocean_df_small['UTC']-ocean_df_small['UTC'][0])
        ocean_df_small['elapsed'] = elapsed_timedelta/np.timedelta64(1, 's')
        
        motion_df_small = pd.read_csv(motion_csv_url, parse_dates = [0])
        print(len(motion_df_small))
        
        # Reindex on timestamp if there are at least a few rows
        if len(ocean_df_small) > 1:
            ocean_df_small.set_index('UTC', drop = True, append = False, inplace = True)
            motion_df_small.set_index('UTC', drop = True, append = False, inplace = True)            
            
            #May need to change this sampling interval:
            sample_interval = '33ms'
            
            ocean_df_small_resample = ocean_df_small.resample(sample_interval).mean()
            motion_df_small_resample = motion_df_small.resample(sample_interval).mean()
            
            # No need to save many extra rows with no fix
            motion_df_small = motion_df_small[~np.isnan(motion_df_small.Latitude)]

            
    else:
        ocean_df_small_resample = pd.DataFrame() # empty DF just so something is returned
        motion_df_small_resample = pd.DataFrame() 
        print(len(motion_df_small_resample))
        return ocean_df_small_resample, motion_df_small_resample

In [34]:
# Fin ID Scraper: 

ride_url_base = 'http://surf.smartfin.org/ride/'
str_id_csv = 'img id="temperatureChart" class="chart" src="' # Look for this text in the HTML contents in fcn below

def read_csv_from_ride2(ride): 
    smartfin_url_base = 'http://surf.smartfin.org'
    ride_url = ride_url_base + str(ride)
    html_page = requests.get(ride_url)
    soup = BeautifulSoup(html_page.text, 'html.parser')
    print(soup)
    links = []
    for link in soup.findAll('a'):
        links.append(link)
    if len(links) > 8: 
        motion_csv_link = str(links[8])
        #print(links[8])
        motion_csv_link = motion_csv_link.split(" ")
        motion_csv_link = motion_csv_link[1]
        motion_csv_link = motion_csv_link.split("\"")
        motion_csv_link = motion_csv_link[1]

        #print(motion_csv_link)

        motion_csv_link = smartfin_url_base + motion_csv_link
        motion_df_small = pd.read_csv(motion_csv_link, parse_dates = [0])

        return 1, motion_df_small
    
    else: 
        
        return 0, 0

In [35]:
appended_motion_list = []
appended_multiIndex = [] # fin_id & ride_id used to identify each DataFrame

counter = 0
index = 1
for ride in ride_ids: 
    #ride = '16167'
    b, motion_df = read_csv_from_ride2(ride)
    # Check to make sure that a .CSV link was found for motion: 
    if b == 1: 
        counter += 1
        print(counter, len(motion_df))
        motion_df.set_index('UTC', drop = True, append = False, inplace = True)
        appended_multiIndex.append(str(ride)) # build list to be multiIndex of future DataFrame
        appended_motion_list.append(motion_df)
    print(index)
    index += 1

print("Total number of rides downloaded: ", counter)
df_keys = tuple(appended_multiIndex)
motion_df = pd.concat(appended_motion_list, keys = df_keys, names = ['ride_id'])

    

<!DOCTYPE html>

<!--[if IE 8]><html class="ie8 no-js" lang="en"><![endif]-->
<!--[if IE 9]><html class="ie9 no-js" lang="en"><![endif]-->
<!--[if !IE]><!-->
<html class="no-js" lang="en">
<!--<![endif]-->
<!-- start: HEAD -->
<head>
<title>Smartfin - 

        Ride 15896
</title>
<meta charset="utf-8"/>
<!--[if IE]><meta http-equiv='X-UA-Compatible' content="IE=edge,IE=9,IE=8,chrome=1" /><![endif]-->
<meta content="width=device-width, initial-scale=1.0, user-scalable=0, minimum-scale=1.0, maximum-scale=1.0" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-web-app-status-bar-style"/>
<meta content="Smartphin Profile Page" name="description">
<meta content="BoardFormula" name="author">
<link href="https://fonts.googleapis.com/css?family=Poppins:700,400,300" rel="stylesheet" type="text/css"/>
<link href="/static/plugins/font-awesome/css/font-awesome.min.css" rel="stylesheet"/>
<link href="/static/plugins/bootstrap/dist/c

1 15029
1
Total number of rides downloaded:  1


In [ ]:
motion_df[:10]

In [12]:
'''
appended_ocean_list = [] # list of DataFrames from original CSVs
appended_motion_list = []
appended_multiIndex = [] # fin_id & ride_id used to identify each DataFrame

print("Once the counter gets to ", len(ride_ids), " it will be done printing.")

## Nested loops (for each fin ID, find all ride IDs, then build a DataFrame from all ride CSVs)
## (Here, ride IDS are either ocean or motion dataframes)
count_good_fins = 0
    
# Loop over ride_ids and find CSVs
counter = 0
for rid in ride_ids:
    counter += 1
    if counter % 10 == 0:
        print(counter)
    try:
        new_motion_df = read_csv_from_ride(rid) # get given ride's CSV from its ride ID using function above
        new_motion_df.set_index('UTC', drop = True, append = False, inplace = True)            

        if not new_motion_df.empty: # Calibration rides, for example
            # Append only if DF isn't empty. There may be a better way to control empty DFs which are created above
            appended_multiIndex.append(str(rid)) # build list to be multiIndex of future DataFrame
            #appended_ocean_list.append(new_ocean_df)
            appended_motion_list.append(new_motion_df)
            #print("Ride data has been uploaded.")
            #print("Ride: ", rid, "data has been uploaded.")
            count_good_fins += 1
            #print("Ride ", rid, "worked!")    
     
    except: 
        print("Ride ", rid, "threw an exception!")    

#%% Build the "Master" DataFrame

# appended_ocean_df.summary()

df_keys = tuple(appended_multiIndex) # keys gotta be a tuple, a list which data in it cannot be changed
#ocean_df = pd.concat(appended_ocean_list, keys = df_keys, names=['ride_id'])
motion_df = pd.concat(appended_motion_list, keys = df_keys, names = ['ride_id'])


##Here, maybe just use info from the motion_df and don't worry about ocean_df data for now.
##If you do want ocean_df data, look at how Phil was getting it from "July 10th and 11th Calibration" jupyter notebook file.
#print(motion_df)
#print(appended_motion_list)
print("Done.")
'''

'\nappended_ocean_list = [] # list of DataFrames from original CSVs\nappended_motion_list = []\nappended_multiIndex = [] # fin_id & ride_id used to identify each DataFrame\n\nprint("Once the counter gets to ", len(ride_ids), " it will be done printing.")\n\n## Nested loops (for each fin ID, find all ride IDs, then build a DataFrame from all ride CSVs)\n## (Here, ride IDS are either ocean or motion dataframes)\ncount_good_fins = 0\n    \n# Loop over ride_ids and find CSVs\ncounter = 0\nfor rid in ride_ids:\n    counter += 1\n    if counter % 10 == 0:\n        print(counter)\n    try:\n        new_motion_df = read_csv_from_ride(rid) # get given ride\'s CSV from its ride ID using function above\n        new_motion_df.set_index(\'UTC\', drop = True, append = False, inplace = True)            \n\n        if not new_motion_df.empty: # Calibration rides, for example\n            # Append only if DF isn\'t empty. There may be a better way to control empty DFs which are created above\n         

In [13]:
# View data in a python dataframe: 
print(len(motion_df))
motion_df[0:10]

# I think that we're resampling each surf session at too small of a rate? 
# Why do we have so little data? 

#Sampling interval of 33ms: 3,290,682 initial data points, 438,099 final data points (adding .033 to UTC time)
#Sampling interval of 20ms: 5,429,605 initial data points, 438,099 final data points (adding .020 to UTC time)

278085


Time  IMU A1  IMU A2  IMU A3  \
ride_id UTC                                                                    
 16302  2019-10-26 19:04:17.967000+00:00  1775440435   498.0    22.0    70.0   
        2019-10-26 19:04:18.217000+00:00  1775440685   499.0    25.0    71.0   
        2019-10-26 19:04:18.469000+00:00  1775440937   498.0    26.0    71.0   
        2019-10-26 19:04:18.719000+00:00  1775441187   498.0    23.0    70.0   
        2019-10-26 19:04:18.970000+00:00  1775441438   508.0    38.0    55.0   
        2019-10-26 19:04:19.224000+00:00  1775441692   498.0    28.0    69.0   
        2019-10-26 19:04:19.475000+00:00  1775441943   497.0    23.0    68.0   
        2019-10-26 19:04:19.726000+00:00  1775442194   499.0    28.0    66.0   
        2019-10-26 19:04:19.975000+00:00  1775442443   499.0    34.0    65.0   
        2019-10-26 19:04:20.217000+00:00  1775442685   497.0    35.0    69.0   

                                          IMU G1  IMU G2  IMU G3  IMU M1  \
ride_id UTC                                                                
 16302  2019-10-26 19:04:17.967000+00:00   -21.0   -10.0     0.0    30.0   
        2019-10-26 19:04:18.217000+00:00   -21.0   -10.0    -5.0    34.0   
        2019-10-26 19:04:18.469000+00:00   -19.0   -10.0    -5.0    40.0   
        2019-10-26 19:04:18.719000+00:00    -9.0    -9.0    -8.0    24.0   
        2019-10-26 19:04:18.970000+00:00   -28.0   -14.0    10.0    37.0   
        2019-10-26 19:04:19.224000+00:00   -21.0    -8.0    12.0    28.0   
        2019-10-26 19:04:19.475000+00:00   -20.0   -10.0    -1.0    28.0   
        2019-10-26 19:04:19.726000+00:00   -21.0    -9.0    -7.0    45.0   
        2019-10-26 19:04:19.975000+00:00   -21.0    -9.0    -4.0    36.0   
        2019-10-26 19:04:20.217000+00:00   -20.0    -9.0    -1.0    31.0   

                                          IMU M2  IMU M3  Latitude  Longitude  
ride_id UTC                                                                    
 16302  2019-10-26 19:04:17.967000+00:00  -168.0  -270.0       NaN        NaN  
        2019-10-26 19:04:18.217000+00:00  -172.0  -276.0       NaN        NaN  
        2019-10-26 19:04:18.469000+00:00  -180.0  -266.0       NaN        NaN  
        2019-10-26 19:04:18.719000+00:00  -174.0  -280.0       NaN        NaN  
        2019-10-26 19:04:18.970000+00:00  -173.0  -275.0       NaN        NaN  
        2019-10-26 19:04:19.224000+00:00  -170.0  -264.0       NaN        NaN  
        2019-10-26 19:04:19.475000+00:00  -180.0  -272.0       NaN        NaN  
        2019-10-26 19:04:19.726000+00:00  -179.0  -267.0       NaN        NaN  
        2019-10-26 19:04:19.975000+00:00  -168.0  -266.0       NaN        NaN  
        2019-10-26 19:04:20.217000+00:00  -173.0  -267.0       NaN        NaN

In [14]:
motion_df.shape

(278085, 12)

In [15]:
# Drop latitude/longitude columns and remove NaN rows: 
motion_df = motion_df.drop(["Latitude", "Longitude"], axis=1)
motion_df = motion_df.dropna(axis=0, how='any')
print(len(motion_df))
motion_df[0:20]

270408


Time  IMU A1  IMU A2  IMU A3  \
ride_id UTC                                                                    
 16302  2019-10-26 19:04:17.967000+00:00  1775440435   498.0    22.0    70.0   
        2019-10-26 19:04:18.217000+00:00  1775440685   499.0    25.0    71.0   
        2019-10-26 19:04:18.469000+00:00  1775440937   498.0    26.0    71.0   
        2019-10-26 19:04:18.719000+00:00  1775441187   498.0    23.0    70.0   
        2019-10-26 19:04:18.970000+00:00  1775441438   508.0    38.0    55.0   
        2019-10-26 19:04:19.224000+00:00  1775441692   498.0    28.0    69.0   
        2019-10-26 19:04:19.475000+00:00  1775441943   497.0    23.0    68.0   
        2019-10-26 19:04:19.726000+00:00  1775442194   499.0    28.0    66.0   
        2019-10-26 19:04:19.975000+00:00  1775442443   499.0    34.0    65.0   
        2019-10-26 19:04:20.217000+00:00  1775442685   497.0    35.0    69.0   
        2019-10-26 19:04:20.467000+00:00  1775442935   499.0    33.0    66.0   
        2019-10-26 19:04:20.719000+00:00  1775443187   498.0    32.0    68.0   
        2019-10-26 19:04:20.969000+00:00  1775443437   499.0    31.0    66.0   
        2019-10-26 19:04:21.219000+00:00  1775443687   498.0    23.0    68.0   
        2019-10-26 19:04:21.470000+00:00  1775443938   487.0    48.0    93.0   
        2019-10-26 19:04:21.721000+00:00  1775444189   519.0    45.0    26.0   
        2019-10-26 19:04:21.972000+00:00  1775444440   484.0     6.0    69.0   
        2019-10-26 19:04:22.222000+00:00  1775444690   494.0    60.0    24.0   
        2019-10-26 19:04:22.474000+00:00  1775444942   505.0    39.0   -21.0   
        2019-10-26 19:04:22.724000+00:00  1775445192   503.0    26.0   -24.0   

                                          IMU G1  IMU G2  IMU G3  IMU M1  \
ride_id UTC                                                                
 16302  2019-10-26 19:04:17.967000+00:00   -21.0   -10.0     0.0    30.0   
        2019-10-26 19:04:18.217000+00:00   -21.0   -10.0    -5.0    34.0   
        2019-10-26 19:04:18.469000+00:00   -19.0   -10.0    -5.0    40.0   
        2019-10-26 19:04:18.719000+00:00    -9.0    -9.0    -8.0    24.0   
        2019-10-26 19:04:18.970000+00:00   -28.0   -14.0    10.0    37.0   
        2019-10-26 19:04:19.224000+00:00   -21.0    -8.0    12.0    28.0   
        2019-10-26 19:04:19.475000+00:00   -20.0   -10.0    -1.0    28.0   
        2019-10-26 19:04:19.726000+00:00   -21.0    -9.0    -7.0    45.0   
        2019-10-26 19:04:19.975000+00:00   -21.0    -9.0    -4.0    36.0   
        2019-10-26 19:04:20.217000+00:00   -20.0    -9.0    -1.0    31.0   
        2019-10-26 19:04:20.467000+00:00   -20.0   -10.0     5.0    28.0   
        2019-10-26 19:04:20.719000+00:00   -22.0   -10.0     7.0    36.0   
        2019-10-26 19:04:20.969000+00:00   -19.0   -10.0     5.0    32.0   
        2019-10-26 19:04:21.219000+00:00   -23.0   -13.0    -1.0    29.0   
        2019-10-26 19:04:21.470000+00:00   -72.0   -53.0   -15.0    30.0   
        2019-10-26 19:04:21.721000+00:00  -254.0   -98.0   -52.0    22.0   
        2019-10-26 19:04:21.972000+00:00   -53.0   -69.0     3.0    26.0   
        2019-10-26 19:04:22.222000+00:00  -126.0   -97.0    23.0     6.0   
        2019-10-26 19:04:22.474000+00:00  -142.0   -64.0    -4.0     2.0   
        2019-10-26 19:04:22.724000+00:00    20.0     9.0    -2.0     5.0   

                                          IMU M2  IMU M3  
ride_id UTC                                               
 16302  2019-10-26 19:04:17.967000+00:00  -168.0  -270.0  
        2019-10-26 19:04:18.217000+00:00  -172.0  -276.0  
        2019-10-26 19:04:18.469000+00:00  -180.0  -266.0  
        2019-10-26 19:04:18.719000+00:00  -174.0  -280.0  
        2019-10-26 19:04:18.970000+00:00  -173.0  -275.0  
        2019-10-26 19:04:19.224000+00:00  -170.0  -264.0  
        2019-10-26 19:04:19.475000+00:00  -180.0  -272.0  
        2019-10-26 19:04:19.726000+00:00  -179.0  -267.0  
        2019-10-26 19:04

### Scrape data from same-day CDIP and add it as an additional column to the dataframe:


In [16]:
# Format should be appending year, month
# Example: Sept. 2019 -> 201909
def create_url_string(year, month):
    url='https://cdip.ucsd.edu/themes/cdip?tz=UTC&numcolorbands=10&palette=cdip_classic&zoom=auto&ll_fmt=dm&high=6.096&r=999&un=1&pb=1&d2=p70&u2=s:201:st:1:v:parameter:dt:'
    if 2014 <= int(year) <= 2019:
        year = year
    if 1 <= int(month) <= 12 and len(month) == 2:
        month = month
    url += year
    url += month
    return url 

In [17]:
# For each row in the dataframe, parse the index ('UTC' column) to get the year, month information. 
motion_df.columns

dates = []
times = []
for row in motion_df.index:
    time = str(row[1])
    date = time.split(" ")
    time = date[1]
    date = date[0]
    time = time.split(".")
    time = time[0]
    time = time.split(":")
    time = time[0] + ":" + time[1]
    times.append(time)
    dates.append(date)
print(dates[0:10], times[0:10])

motion_df["Date"] = dates
motion_df["Time"] = times

['2019-10-26', '2019-10-26', '2019-10-26', '2019-10-26', '2019-10-26', '2019-10-26', '2019-10-26', '2019-10-26', '2019-10-26', '2019-10-26'] ['19:04', '19:04', '19:04', '19:04', '19:04', '19:04', '19:04', '19:04', '19:04', '19:04']


In [18]:
motion_df[0:10]

Time  IMU A1  IMU A2  IMU A3  \
ride_id UTC                                                               
 16302  2019-10-26 19:04:17.967000+00:00  19:04   498.0    22.0    70.0   
        2019-10-26 19:04:18.217000+00:00  19:04   499.0    25.0    71.0   
        2019-10-26 19:04:18.469000+00:00  19:04   498.0    26.0    71.0   
        2019-10-26 19:04:18.719000+00:00  19:04   498.0    23.0    70.0   
        2019-10-26 19:04:18.970000+00:00  19:04   508.0    38.0    55.0   
        2019-10-26 19:04:19.224000+00:00  19:04   498.0    28.0    69.0   
        2019-10-26 19:04:19.475000+00:00  19:04   497.0    23.0    68.0   
        2019-10-26 19:04:19.726000+00:00  19:04   499.0    28.0    66.0   
        2019-10-26 19:04:19.975000+00:00  19:04   499.0    34.0    65.0   
        2019-10-26 19:04:20.217000+00:00  19:04   497.0    35.0    69.0   

                                          IMU G1  IMU G2  IMU G3  IMU M1  \
ride_id UTC                                                                
 16302  2019-10-26 19:04:17.967000+00:00   -21.0   -10.0     0.0    30.0   
        2019-10-26 19:04:18.217000+00:00   -21.0   -10.0    -5.0    34.0   
        2019-10-26 19:04:18.469000+00:00   -19.0   -10.0    -5.0    40.0   
        2019-10-26 19:04:18.719000+00:00    -9.0    -9.0    -8.0    24.0   
        2019-10-26 19:04:18.970000+00:00   -28.0   -14.0    10.0    37.0   
        2019-10-26 19:04:19.224000+00:00   -21.0    -8.0    12.0    28.0   
        2019-10-26 19:04:19.475000+00:00   -20.0   -10.0    -1.0    28.0   
        2019-10-26 19:04:19.726000+00:00   -21.0    -9.0    -7.0    45.0   
        2019-10-26 19:04:19.975000+00:00   -21.0    -9.0    -4.0    36.0   
        2019-10-26 19:04:20.217000+00:00   -20.0    -9.0    -1.0    31.0   

                                          IMU M2  IMU M3        Date  
ride_id UTC                                                           
 16302  2019-10-26 19:04:17.967000+00:00  -168.0  -270.0  2019-10-26  
        2019-10-26 19:04:18.217000+00:00  -172.0  -276.0  2019-10-26  
        2019-10-26 19:04:18.469000+00:00  -180.0  -266.0  2019-10-26  
        2019-10-26 19:04:18.719000+00:00  -174.0  -280.0  2019-10-26  
        2019-10-26 19:04:18.970000+00:00  -173.0  -275.0  2019-10-26  
        2019-10-26 19:04:19.224000+00:00  -170.0  -264.0  2019-10-26  
        2019-10-26 19:04:19.475000+00:00  -180.0  -272.0  2019-10-26  
        2019-10-26 19:04:19.726000+00:00  -179.0  -267.0  2019-10-26  
        2019-10-26 19:04:19.975000+00:00  -168.0  -266.0  2019-10-26  
        2019-10-26 19:04:20.217000+00:00  -173.0  -267.0  2019-10-26

In [19]:
# Define a function that will find the closest time on the CDIP table to the surf ride's time: 
def find_closest_single_time(date, time_utc):
    time_list = []
    
    # Iterate over each date
    for x in df_data['Date(UTC)']:
        a = x.split(" ")
        if date == a[0]:
            a = str(a[1]).split(':')
            y = str(time_utc).split(":")
            #print(a)
            #print(y)
            
            # Find the closest timestamp (less than 15 minutes away)
            time1 = int(a[0])*60 + int(a[1])
            time2 = int(y[0])*60 + int(y[1])
            
            if abs(time1 - time2) <= 15:
                time_string = str(a[0]) + ":" + str(a[1])
                time_list.append(time_string)
           
    return time_list

### Create the output labels from CDIP data for each Smartfin timestamp: 


In [20]:
# Create the output labels from CDIP data for each Smartfin timestamp: 
old_url = 'old_url'
old_date = 'old_date'
old_time = 'xx:xx'

wave_heights = []
wave_periods = []
wave_directions = []
for d, time in zip(dates,times):
    di = d.split("-")
    year = di[0]
    month = di[1]
    url = create_url_string(year, month)
    
    #We need to load a new table since we have a new surf ride session:
    if (url != old_url): 
        old_url = url
        # Create a handle, page, to handle the contents of the website
        page = requests.get(url)

        # Store the contents of the website under doc
        doc = lh.fromstring(page.content)

        # Parse data stored between <tr>..</tr> of HTML
        tr_elements = doc.xpath('//tr')

        # Need to drop the first 3 rows since they aren't in the table
        tr_elements = tr_elements[3:]
        
        # Parse the first row as the header
        tr_elements = doc.xpath('//tr')

        # Create empty list
        headers = []
        i = 0

        # For each row, store each first element (header) and an empty list
        for t in tr_elements[3]:
            i+=1
            name=t.text_content()
            #print(name)
            headers.append(name)

        #print(headers)

        # Create a Pandas dataframe: 
        data_list = []

        #Since out first row is the header, data is stored on the second row onwards
        for j in range(4, len(tr_elements)):
            data = parse_rows(j)
            data_list.append(data)

        df_data = pd.DataFrame(data_list, columns=headers)
        #print(df_data[:10])
        
        #Find the closest time in the CDIP data table that corresponds to the Smartfin data: 
        date = d #YYYY-MM-DD format
        print("date, time ", date, time)
        time_list = find_closest_single_time(date, time)
        print("time list ", time_list)
        
        # Now compute the average significant wave height for that time period:
        length = df_data['Date(UTC)'].size
        wave_height_list = []
        wave_period_list = []
        wave_direction_list = []
        for time in time_list: 
            date_data = date + " " + time
            for i in range(0, length):
                if df_data['Date(UTC)'][i] == date_data:
                    print("height ", df_data['Hs(ft)'][i])
                    wave_height_list.append(float(df_data['Hs(ft)'][i]))
                    wave_period_list.append(float(df_data['Tp(s)'][i]))
                    wave_direction_list.append(float(df_data['Dp(deg)'][i]))
                    
        # Throws error when date doesn't exist on CDIP data (ex: 2018-01-18)
        if len(wave_height_list) == 0 or len(wave_period_list) == 0 or len(wave_direction_list) == 0: 
            avg_wave_height = np.nan
            avg_wave_period = np.nan
            avg_wave_direction = np.nan
            
        else: 
            #print(wave_height_list)
            avg_wave_height = sum(wave_height_list)/len(wave_height_list)
            avg_wave_period = sum(wave_period_list)/len(wave_period_list)
            avg_wave_direction = sum(wave_direction_list)/len(wave_direction_list)
            #print(avg_wave_height)
            #print(avg_wave_period)
            #print(avg_wave_direction) 
            
    # Same date/month but maybe a different day or time:
    elif old_date != date or old_time[3] != time[3]:
        date = d #YYYY-MM-DD format
        #print(date, time)
        time_list = find_closest_single_time(date, time)
        #print(time_list)
        
        old_date = date
        old_time = time
        
        # Now compute the average significant wave height for that time period:
        length = df_data['Date(UTC)'].size
        wave_height_list = []
        wave_period_list = []
        wave_direction_list = []
        for time in time_list: 
            date_data = date + " " + time
            for i in range(0, length):
                if df_data['Date(UTC)'][i] == date_data:
                    #print(df_data['Hs(ft)'][i])
                    wave_height_list.append(float(df_data['Hs(ft)'][i]))
                    wave_period_list.append(float(df_data['Tp(s)'][i]))
                    wave_direction_list.append(float(df_data['Dp(deg)'][i]))
                    
      
        # Throws error when date doesn't exist on CDIP data (ex: 2018-01-18)
        if len(wave_height_list) == 0 or len(wave_period_list) == 0 or len(wave_direction_list) == 0: 
            avg_wave_height = np.nan
            avg_wave_period = np.nan
            avg_wave_direction = np.nan
            
        else: 
            #print(wave_height_list)
            avg_wave_height = sum(wave_height_list)/len(wave_height_list)
            avg_wave_period = sum(wave_period_list)/len(wave_period_list)
            avg_wave_direction = sum(wave_direction_list)/len(wave_direction_list)
            #print(avg_wave_height)
            #print(avg_wave_period)
            #print(avg_wave_direction)    

    # If they have the same url, date, and time then they will have the same value computed. 
    wave_heights.append(avg_wave_height)
    wave_periods.append(avg_wave_period)
    wave_directions.append(avg_wave_direction)

date, time  2019-10-26 19:04
time list  ['19:02']
height  1.25
date, time  2019-09-24 15:28
time list  ['15:32']
height  1.80
date, time  2019-02-25 01:31
time list  ['01:32']
height  1.71
date, time  2018-01-18 14:17
time list  []
date, time  2017-05-03 18:40
time list  ['18:51']
height  1.80


In [21]:
print(len(dates))
print(len(wave_heights))
print(len(wave_periods))
print(len(wave_directions))


# Check to make sure different wave heights were appended throughout the dataframe: 
motion_df["Hs(ft)"] = wave_heights
motion_df["Tp(s)"] = wave_periods
motion_df["Dp(deg)"] = wave_directions

for x in range(0, 60000, 10000):
    print(motion_df.iloc[[x]]['Hs(ft)'])
    print(motion_df.iloc[[x]]['Tp(s)'])
    print(motion_df.iloc[[x]]['Dp(deg)'])

motion_df[0:5]

270408
270408
270408
270408
ride_id  UTC                             
 16302   2019-10-26 19:04:17.967000+00:00    1.25
Name: Hs(ft), dtype: float64
ride_id  UTC                             
 16302   2019-10-26 19:04:17.967000+00:00    9.09
Name: Tp(s), dtype: float64
ride_id  UTC                             
 16302   2019-10-26 19:04:17.967000+00:00    284.0
Name: Dp(deg), dtype: float64
ride_id  UTC                             
 16302   2019-10-26 19:44:24.676000+00:00    1.35
Name: Hs(ft), dtype: float64
ride_id  UTC                             
 16302   2019-10-26 19:44:24.676000+00:00    9.09
Name: Tp(s), dtype: float64
ride_id  UTC                             
 16302   2019-10-26 19:44:24.676000+00:00    285.0
Name: Dp(deg), dtype: float64
ride_id  UTC                             
 16301   2019-10-25 19:17:02.984000+00:00    2.0
Name: Hs(ft), dtype: float64
ride_id  UTC                             
 16301   2019-10-25 19:17:02.984000+00:00    9.09
Name: Tp(s), dtype: float64
ride

Time  IMU A1  IMU A2  IMU A3  \
ride_id UTC                                                               
 16302  2019-10-26 19:04:17.967000+00:00  19:04   498.0    22.0    70.0   
        2019-10-26 19:04:18.217000+00:00  19:04   499.0    25.0    71.0   
        2019-10-26 19:04:18.469000+00:00  19:04   498.0    26.0    71.0   
        2019-10-26 19:04:18.719000+00:00  19:04   498.0    23.0    70.0   
        2019-10-26 19:04:18.970000+00:00  19:04   508.0    38.0    55.0   

                                          IMU G1  IMU G2  IMU G3  IMU M1  \
ride_id UTC                                                                
 16302  2019-10-26 19:04:17.967000+00:00   -21.0   -10.0     0.0    30.0   
        2019-10-26 19:04:18.217000+00:00   -21.0   -10.0    -5.0    34.0   
        2019-10-26 19:04:18.469000+00:00   -19.0   -10.0    -5.0    40.0   
        2019-10-26 19:04:18.719000+00:00    -9.0    -9.0    -8.0    24.0   
        2019-10-26 19:04:18.970000+00:00   -28.0   -14.0    10.0    37.0   

                                          IMU M2  IMU M3        Date  Hs(ft)  \
ride_id UTC                                                                    
 16302  2019-10-26 19:04:17.967000+00:00  -168.0  -270.0  2019-10-26    1.25   
        2019-10-26 19:04:18.217000+00:00  -172.0  -276.0  2019-10-26    1.25   
        2019-10-26 19:04:18.469000+00:00  -180.0  -266.0  2019-10-26    1.25   
        2019-10-26 19:04:18.719000+00:00  -174.0  -280.0  2019-10-26    1.25   
        2019-10-26 19:04:18.970000+00:00  -173.0  -275.0  2019-10-26    1.25   

                                          Tp(s)  Dp(deg)  
ride_id UTC                                               
 16302  2019-10-26 19:04:17.967000+00:00   9.09    284.0  
        2019-10-26 19:04:18.217000+00:00   9.09    284.0  
        2019-10-26 19:04:18.469000+00:00   9.09    284.0  
        2019-10-26 19:04:18.719000+00:00   9.09    284.0  
        2019-10-26 19:04:18.970000+00:00   9.09    284.0

In [22]:
# Now drop all of the nan values that I introduced earlier: 
motion_df = motion_df.dropna(axis=0, how='any')
print(len(motion_df))
motion_df[0:5]

159465


Time  IMU A1  IMU A2  IMU A3  \
ride_id UTC                                                               
 16302  2019-10-26 19:04:17.967000+00:00  19:04   498.0    22.0    70.0   
        2019-10-26 19:04:18.217000+00:00  19:04   499.0    25.0    71.0   
        2019-10-26 19:04:18.469000+00:00  19:04   498.0    26.0    71.0   
        2019-10-26 19:04:18.719000+00:00  19:04   498.0    23.0    70.0   
        2019-10-26 19:04:18.970000+00:00  19:04   508.0    38.0    55.0   

                                          IMU G1  IMU G2  IMU G3  IMU M1  \
ride_id UTC                                                                
 16302  2019-10-26 19:04:17.967000+00:00   -21.0   -10.0     0.0    30.0   
        2019-10-26 19:04:18.217000+00:00   -21.0   -10.0    -5.0    34.0   
        2019-10-26 19:04:18.469000+00:00   -19.0   -10.0    -5.0    40.0   
        2019-10-26 19:04:18.719000+00:00    -9.0    -9.0    -8.0    24.0   
        2019-10-26 19:04:18.970000+00:00   -28.0   -14.0    10.0    37.0   

                                          IMU M2  IMU M3        Date  Hs(ft)  \
ride_id UTC                                                                    
 16302  2019-10-26 19:04:17.967000+00:00  -168.0  -270.0  2019-10-26    1.25   
        2019-10-26 19:04:18.217000+00:00  -172.0  -276.0  2019-10-26    1.25   
        2019-10-26 19:04:18.469000+00:00  -180.0  -266.0  2019-10-26    1.25   
        2019-10-26 19:04:18.719000+00:00  -174.0  -280.0  2019-10-26    1.25   
        2019-10-26 19:04:18.970000+00:00  -173.0  -275.0  2019-10-26    1.25   

                                          Tp(s)  Dp(deg)  
ride_id UTC                                               
 16302  2019-10-26 19:04:17.967000+00:00   9.09    284.0  
        2019-10-26 19:04:18.217000+00:00   9.09    284.0  
        2019-10-26 19:04:18.469000+00:00   9.09    284.0  
        2019-10-26 19:04:18.719000+00:00   9.09    284.0  
        2019-10-26 19:04:18.970000+00:00   9.09    284.0

In [23]:
# We're collecting IMU data at 3-4 Hz here. 
# We're using 4 surf sessions and we have 60,000 data points. 
# If we only calculated Hs once per minute then we would have 330 data points. 

# Instead of looking at significant wave height, we could look at wave direction 
# or we could look at Hs, dir, and period and see which one we predict best. 



In [24]:
# Convert raw IMU values to real values: 
g_const = 512 #Raw acceleration constant 512 = 1g (accelerometer's measured force due to gravity)
gravity = 9.80665 #Approximate measurement for gravity

# Correct the IMU Acceleration columns into units of meters
# Dividing by 512 is equivalent to muliplying by 4 to correct the bit shifting by 2 places and dividing by 2048 to convert bits to G's
# Multiplying by the 9.81 afterwards is simply to convert G's into m/s^2
motion_df['IMU A1'] = motion_df['IMU A1'].apply(lambda x: x / g_const * gravity)
motion_df['IMU A2'] = motion_df['IMU A2'].apply(lambda x: x / g_const * gravity)
motion_df['IMU A3'] = motion_df['IMU A3'].apply(lambda x: x / g_const * gravity)

In [25]:
# Gyroscopic Rotation converts to deg/s
gyro_const = 8.2 # Raw gyrscope constant 8.2 bits per degree
motion_df['IMU G1'] = motion_df['IMU G1'].apply(lambda x: x / gyro_const)
motion_df['IMU G2'] = motion_df['IMU G2'].apply(lambda x: x / gyro_const)
motion_df['IMU G3'] = motion_df['IMU G3'].apply(lambda x: x / gyro_const)

In [26]:
# Magnetometer values
# Offset variables help in recentering the magnetic data in order to define direction and use trig functions
'''
M1_offset_var = 219.786
M2_offset_var = 180
M3_offset_var = 280

motion_df['IMU M1'] = motion_df['IMU M1'].apply(lambda x: x - M1_offset_var)
motion_df['IMU M2'] = motion_df['IMU M2'].apply(lambda x: x - M2_offset_var)
motion_df['IMU M3'] = motion_df['IMU M3'].apply(lambda x: x - M3_offset_var)
'''

"\nM1_offset_var = 219.786\nM2_offset_var = 180\nM3_offset_var = 280\n\nmotion_df['IMU M1'] = motion_df['IMU M1'].apply(lambda x: x - M1_offset_var)\nmotion_df['IMU M2'] = motion_df['IMU M2'].apply(lambda x: x - M2_offset_var)\nmotion_df['IMU M3'] = motion_df['IMU M3'].apply(lambda x: x - M3_offset_var)\n"

In [27]:
print(len(motion_df))
motion_df[:20]

159465


Time    IMU A1    IMU A2    IMU A3  \
ride_id UTC                                                                     
 16302  2019-10-26 19:04:17.967000+00:00  19:04  9.538499  0.421379  1.340753   
        2019-10-26 19:04:18.217000+00:00  19:04  9.557653  0.478840  1.359907   
        2019-10-26 19:04:18.469000+00:00  19:04  9.538499  0.497994  1.359907   
        2019-10-26 19:04:18.719000+00:00  19:04  9.538499  0.440533  1.340753   
        2019-10-26 19:04:18.970000+00:00  19:04  9.730036  0.727837  1.053449   
        2019-10-26 19:04:19.224000+00:00  19:04  9.538499  0.536301  1.321599   
        2019-10-26 19:04:19.475000+00:00  19:04  9.519346  0.440533  1.302446   
        2019-10-26 19:04:19.726000+00:00  19:04  9.557653  0.536301  1.264138   
        2019-10-26 19:04:19.975000+00:00  19:04  9.557653  0.651223  1.244985   
        2019-10-26 19:04:20.217000+00:00  19:04  9.519346  0.670376  1.321599   
        2019-10-26 19:04:20.467000+00:00  19:04  9.557653  0.632069  1.264138   
        2019-10-26 19:04:20.719000+00:00  19:04  9.538499  0.612916  1.302446   
        2019-10-26 19:04:20.969000+00:00  19:04  9.557653  0.593762  1.264138   
        2019-10-26 19:04:21.219000+00:00  19:04  9.538499  0.440533  1.302446   
        2019-10-26 19:04:21.470000+00:00  19:04  9.327810  0.919373  1.781286   
        2019-10-26 19:04:21.721000+00:00  19:04  9.940725  0.861913  0.497994   
        2019-10-26 19:04:21.972000+00:00  19:04  9.270349  0.114922  1.321599   
        2019-10-26 19:04:22.222000+00:00  19:04  9.461885  1.149217  0.459687   
        2019-10-26 19:04:22.474000+00:00  19:04  9.672575  0.746991 -0.402226   
        2019-10-26 19:04:22.724000+00:00  19:04  9.634267  0.497994 -0.459687   

                                             IMU G1     IMU G2    IMU G3  \
ride_id UTC                                                                
 16302  2019-10-26 19:04:17.967000+00:00  -2.560976  -1.219512  0.000000   
        2019-10-26 19:04:18.217000+00:00  -2.560976  -1.219512 -0.609756   
        2019-10-26 19:04:18.469000+00:00  -2.317073  -1.219512 -0.609756   
        2019-10-26 19:04:18.719000+00:00  -1.097561  -1.097561 -0.975610   
        2019-10-26 19:04:18.970000+00:00  -3.414634  -1.707317  1.219512   
        2019-10-26 19:04:19.224000+00:00  -2.560976  -0.975610  1.463415   
        2019-10-26 19:04:19.475000+00:00  -2.439024  -1.219512 -0.121951   
        2019-10-26 19:04:19.726000+00:00  -2.560976  -1.097561 -0.853659   
        2019-10-26 19:04:19.975000+00:00  -2.560976  -1.097561 -0.487805   
        2019-10-26 19:04:20.217000+00:00  -2.439024  -1.097561 -0.121951   
        2019-10-26 19:04:20.467000+00:00  -2.439024  -1.219512  0.609756   
        2019-10-26 19:04:20.719000+00:00  -2.682927  -1.219512  0.853659   
        2019-10-26 19:04:20.969000+00:00  -2.317073  -1.219512  0.609756   
        2019-10-26 19:04:21.219000+00:00  -2.804878  -1.585366 -0.121951   
        2019-10-26 19:04:21.470000+00:00  -8.780488  -6.463415 -1.829268   
        2019-10-26 19:04:21.721000+00:00 -30.975610 -11.951220 -6.341463   
        2019-10-26 19:04:21.972000+00:00  -6.463415  -8.414634  0.365854   
        2019-10-26 19:04:22.222000+00:00 -15.365854 -11.829268  2.804878   
        2019-10-26 19:04:22.474000+00:00 -17.317073  -7.804878 -0.487805   
        2019-10-26 19:04:22.724000+00:00   2.439024   1.097561 -0.243902   

                                          IMU M1  IMU M2  IMU M3        Date  \
ride_id UTC                                                                    
 16302  2019-10-26 19:04:17.967000+00:00    30.0  -168.0  -270.0  2019-10-26   
        2019-10-26 19:04:18.217000+00:00    34.0  -172.0  -276.0  2019-10-26   
        2019-10-26 19:04:18.469000+00:00    40.0  -180.0  -266.0  2019-10-26   
        2019-10-26 19:04:18.719000+00:00    24.0  -174.0  -280.0  2019-10-26   
        2019-10-26 19:04:18.970000+00:00    37.0  -173.0  -275.0  2019-10-26   
        2019-10-26 19:04:19

In [28]:
# After converting all to real values, try to export data to .CSV so everyone else doesn't have to webscrape it:
#motion_df.to_csv('CSE258_A2_Data.csv')

In [29]:
#motion_df.to_csv('CSE258_A2_Data2.csv')